# Fall 2023

In [46]:
# The following snippet will automatically reload imported code in helpers.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import csv
from datetime import date, time, timedelta
from typing import Dict, List

from helpers import calendar_table, new_syllabus, show_calendar, Plan, meeting_days, Kind, cut, schedule, merge, serialize, generate_VL_index_page, make_VL_page
from helpers import JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC, MON, WED, FRI, TUES, THUR

## Key Dates

In [48]:
YEAR = 2024

FDOC = date(YEAR, MAY, 15)

GRADESCOPE = "https://www.gradescope.com/"

# SPRING HOLIDAYS
# HOLIDAYS = {
#     date(YEAR, JAN, 15),     # MLK JR Day
#     date(YEAR, FEB, 12),     # Well-being Day
#     date(YEAR, FEB, 13),    # Well-being Day
#     date(YEAR, MAR, 28),    # Well-being Day
#     date(YEAR, MAR, 11),    # Spring Break
#     date(YEAR, MAR, 12),    # Spring Break
#     date(YEAR, MAR, 13),    # Spring Break
#     date(YEAR, MAR, 14),    # Spring Break
#     date(YEAR, MAR, 15),    # Spring Break
#     date(YEAR, MAR, 29),    # Univ. Holiday
    
# }


# SUMMER SESSION 1 HOLIDAYS
HOLIDAYS = {
    date(YEAR, MAY, 27),     # Memorial Day
    #date(YEAR, JUN, 19),     # Juneteenth
}


# #FALL HOLIDAYS 
# HOLIDAYS = {
#     date(YEAR, SEP, 4),     # Labor Day
#     date(YEAR, OCT, 12),    # University Day 
#     date(YEAR, SEP, 5),     # Well-being Day
#     date(YEAR, SEP, 25),    # Well-being Day
#     date(YEAR, OCT, 19),    # Fall Break
#     date(YEAR, OCT, 20),    # Fall Break
#     date(YEAR, NOV, 22),    # Thanksgiving Break
#     date(YEAR, NOV, 23),    # Thanksgiving Break
#     date(YEAR, NOV, 24),    # Thanksgiving Break
# }

DUKE_GAMES = {}

DROP_PERIOD_ENDS = date(YEAR, MAY, 16)
WITHDRAW_PERIOD_ENDS = date(YEAR, MAY, 16)

LDOC = date(YEAR, JUN, 18)

MEETING_DAYS = { MON, TUES, WED, THUR, FRI }

START_TIME = time(9, 45, 0)
END_TIME = time(11, 15, 0)

FINALS = [date(YEAR, JUN, 21)]
FINALS_TIMES = [time(8, 0, 0)]

TIMESPAN = (START_TIME, END_TIME)

## Skeleton

Let's start from the skeleton of a semester with holidays, important semester milestones, Duke games, and lecture days pencilled in.

In [49]:
syllabus: Dict[date, List[Plan]] = new_syllabus(FDOC, LDOC)

for holiday in HOLIDAYS:
    syllabus[holiday].append(Plan("University Holiday", Kind.NA, 0, holiday))

for duke_game in DUKE_GAMES:
    syllabus[duke_game].append(Plan("Duke Game", Kind.NA, 0, duke_game))

syllabus[DROP_PERIOD_ENDS].append(Plan("Drop Period Ends", Kind.NA, 0, DROP_PERIOD_ENDS)) 
syllabus[WITHDRAW_PERIOD_ENDS].append(Plan("Widthdraw Period Ends", Kind.NA, 0, WITHDRAW_PERIOD_ENDS))

MEETINGS = meeting_days(FDOC, LDOC, MEETING_DAYS, HOLIDAYS)
LECTURES = []
for date_idx in range(len(MEETINGS)):
        day = MEETINGS[date_idx]
        lecture = Plan(f"Day {date_idx:02}", Kind.CL, date_idx, day)
        LECTURES.append(lecture)
        syllabus[day].append(lecture)

FN = []
for final_idx in range(len(FINALS)):
    final_time = FINALS_TIMES[final_idx].strftime("%-I:%M%p")
    FN.append(schedule(f"Final @ {final_time}", syllabus, Kind.FN, 0, FINALS[final_idx]))

table = calendar_table(syllabus)
with open('schedule.csv', "w") as csvfile:
    for row in table:
        print(",".join(row))
        csvfile.write(",".join(row) + "\n")


Date,Weekday,Plan
Week 0
May-15,Wed,FDOC
,,Day 00
May-16,Thu,Drop Period Ends
,,Widthdraw Period Ends
,,Day 01
May-17,Fri,Day 02
Week 1
May-20,Mon,Day 03
May-21,Tue,Day 04
May-22,Wed,Day 05
May-23,Thu,Day 06
May-24,Fri,Day 07
Week 2
May-27,Mon,University Holiday
May-28,Tue,Day 08
May-29,Wed,Day 09
May-30,Thu,Day 10
May-31,Fri,Day 11
Week 3
Jun-03,Mon,Day 12
Jun-04,Tue,Day 13
Jun-05,Wed,Day 14
Jun-06,Thu,Day 15
Jun-07,Fri,Day 16
Week 4
Jun-10,Mon,Day 17
Jun-11,Tue,Day 18
Jun-12,Wed,Day 19
Jun-13,Thu,Day 20
Jun-14,Fri,Day 21
Week 5
Jun-17,Mon,Day 22
Jun-18,Tue,LDOC
,,Day 23
Jun-21,Fri,Final @ 8:00AM


## Quizzes

In [50]:
# Update the following tuple to reflect meeting days quizzes are held on.
QZ_MEETING_DAYS = (7, 11, 16)
#QZ_MEETING_DAYS = (5, 11, 17, 23)

quizzes = cut(syllabus, FDOC, LDOC)

QZ: Plan = list()


for i in range(len(QZ_MEETING_DAYS)):
    quiz_date = MEETINGS[QZ_MEETING_DAYS[i]]
    QZ.append(schedule(f"Quiz {i}", quizzes, Kind.QZ, i, quiz_date, quiz_date))
#show_calendar(quizzes)

In [51]:

#TODO: For Sp24, change back to end first unit at QZ[0]
UNIT_DATES = [
    { "start": FDOC, "end": QZ[1].date },
    { "start": QZ[0].date + timedelta(days=1), "end": QZ[1].date },
    { "start": QZ[1].date + timedelta(days=1), "end": QZ[2].date },
    { "start": QZ[2].date + timedelta(days=1), "end": FN[-1].date }
]

unit: list[dict[date, list[Plan]]] = []
for i in range(len(UNIT_DATES)):
    unit_dates = UNIT_DATES[i]
    unit.append(cut(syllabus, UNIT_DATES[i]["start"], UNIT_DATES[i]["end"]))


In [52]:
# Collections for various components of the course
CL: list[Plan] = []
LS: list[Plan] = []
EX: list[Plan] = []
CQ: list[Plan] = []
RD: list[Plan] = []
VL: list[Plan] = []

# These indices are kind of a strange idea at first glance. The problem we are hoping to solve is
# keeping track of index numbers (to count up indices of each course component), but we'd like 
# to be able to reexecute unit cells below. So what we do is keep an index for each of the units.
# When subsequent units begin, they start from the count of the previous range (and 0 in the first).
# This way, you can reexecute cells below multiple times without having to reinitialize these indices.
# This is related to the trick of cutting / pasting the lesson plans.
CL_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
LS_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
EX_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
CQ_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
RD_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
VL_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]

LS_due_delta: timedelta = timedelta(days=0)
VL_due_delta: timedelta = timedelta(days=1)
CQ_due_delta: timedelta = timedelta(days=0)
EX_due_delta: timedelta = timedelta(days=6)
RD_due_delta: timedelta = timedelta(days=20)

LS_VIDEOS = []

LDOC = MEETINGS[len(MEETINGS)-1]
# Auto populate LS
for idx in range(100):
    LS.append(schedule("", unit[0], Kind.LS, LS_idxs[0], LDOC))


In [53]:
u = 0  # Current unit index

# Reset stateful counters and unit schedule
CL_idxs[u] = 0 # First unit starts CL from 0
CL = CL[:CL_idxs[u]] # Reset CL list - redundant in first unit, but keeping for consistency
VL_idxs[u] = 0 # First unit starts VL from 0
VL = VL[:VL_idxs[u]] # Reset VL list - redundant in first unit, but keeping for consistency
LS_idxs[u] = 0 # First unit starts LS from 0
#LS = LS[:LS_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
EX_idxs[u] = 0 # First unit starts EX from 0
EX = EX[:EX_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
CQ_idxs[u] = 0 # First unit starts CQ from 0
#CQ = CQ[:CQ_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
RD_idxs[u] = 0 # First unit starts CQ from 0
RD = RD[:RD_idxs[u]] # Reset RD list - redundant in first unit, but keeping for consistency
unit[u] = cut(syllabus, UNIT_DATES[u]["start"], UNIT_DATES[u]["end"])

# Date Context is the current Meeting # we're anchoring off of. We make this a variable to make
# moving around scheduled items easier.

### Tues, Aug 16
date_ctx = MEETINGS[0]

CL.append(schedule("Welcome to COMP110 - An Intro to the Class", unit[u], Kind.CL, CL_idxs[u], date_ctx))
CL[CL_idxs[u]].links = {
    "Course Intro Slides": "/static/slides/SSI-CL00.pdf",
    "Setup Part 0: Update Operating System": "/resources/setup/os-update.html",
    "Setup Part 1: Install Needed Software": "/resources/setup/software.html",
    "Setup Part 2: Setup your Workspace" : "/resources/setup/workspace.html"
}
CL_idxs[u] += 1


LS[LS_idxs[u]] = (schedule("Syllabus and Support", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta))
LS[LS_idxs[u]].links = {
     "1. Read the Syllabus": "/resources/syllabus.html",
     "2. Read Support Resources": "/support",
     "3. Complete Lesson 00 Responses on Gradescope": "https://www.gradescope.com/"
}
#LS[LS_idxs[u]].add_extension(timedelta(days=5))

LS_idxs[u] += 1

LS_VIDEOS.append(["Syllabus and Support",{"syllabus":"/resources/syllabus.html"}])


EX.append(schedule("Hello World", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + timedelta(days=1) ))
EX[EX_idxs[u]].links = {
    "": '/exercises/ex00_hello_world.html'
}
#EX[EX_idxs[u]].add_extension(timedelta(days=8))
EX_idxs[u] += 1



date_ctx = MEETINGS[1]

CL.append(schedule("An Introduction to Coding + Beginning Concepts", unit[u], Kind.CL, CL_idxs[u], date_ctx))
CL[CL_idxs[u]].links = {
    "Lecture Slides": "/static/slides/SS1-CL01.pdf",
    "Objects + Data Types Lecture Notes": "/lessons/objects-data-types.html",
    "Expressions Lecture Notes": "/lessons/expressions.html"
}
CL_idxs[u] += 1

LS[LS_idxs[u]] = (schedule("Objects and Data Types", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta ))
LS[LS_idxs[u]].links = {
     "": GRADESCOPE
}
LS_idxs[u] += 1

LS[LS_idxs[u]] = (schedule("Expressions", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta ))
LS[LS_idxs[u]].links = {
     "": GRADESCOPE
}
LS_idxs[u] += 1

LS[LS_idxs[u]] = (schedule("Variables and User Input", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta))
LS[LS_idxs[u]].links = {
     "": GRADESCOPE
}
LS_idxs[u] += 1

date_ctx = MEETINGS[2]

CL.append(schedule("Boolean Operators + Conditionals", unit[u], Kind.CL, CL_idxs[u], date_ctx))
CL[CL_idxs[u]].links = {
    "Lecture Slides": "/static/slides/SS1-CL02.pdf"
}
CL_idxs[u] += 1

LS[LS_idxs[u]] = (schedule("Boolean Operators", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta))
LS[LS_idxs[u]].links = {
     "": GRADESCOPE
}
LS_idxs[u] += 1

LS[LS_idxs[u]] = (schedule("while Loop Statements", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta))
LS[LS_idxs[u]].links = {
     "": GRADESCOPE
}
LS_idxs[u] += 1


# # VL.append(schedule("Virtual Lesson", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# # VL[VL_idxs[u]].links = {
# #     "": "/virtual-classes/VL00.html",
# # }
# lessons = [["VS Code Walkthrough", { "Video": "https://youtu.be/uJZs73924ls", "Assignment": "https://www.gradescope.com/"}],
#            ["Objects and Data Types", {"Notes" : "/lessons/objects-data-types.html", "Video" : "https://youtu.be/6GxkRgIze-c", "Assignment": "https://www.gradescope.com/"}],
#            ["Expressions", {"Notes": "/lessons/expressions.html", "Assignment": "https://www.gradescope.com/"}],
#            ["Variables and User Input", {"Video" : "https://youtu.be/GPpYSDNUtH8", "Assignment": "https://www.gradescope.com/"}]]
# # make_VL_page(lessons,[], VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])

# # VL[VL_idxs[u]].add_extension(timedelta(days=13))
# # VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)


# date_ctx = MEETINGS[2]

# CL.append(schedule("Variables, Expressions, and User Input ", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/TwT8oBikCQE?feature=share",
#     "Lecture Slides": "/static/slides/CL02.pdf",
#     "Lecture Notes": "/lessons/expressions.html"
# }
# CL_idxs[u] += 1

# LS[LS_idxs[u]] =(schedule("Objects and Types", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta + timedelta(days=2)))
# LS[LS_idxs[u]].links = {
#      "": GRADESCOPE
# }

# LS_idxs[u] += 1


# LS[LS_idxs[u]] = (schedule("Expressions", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta + timedelta(days=2)))
# LS[LS_idxs[u]].links = {
#      "": GRADESCOPE
# }

# LS_idxs[u] += 1


# LS[LS_idxs[u]] =(schedule("Variables and User Input", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta + timedelta(days=2)))
# LS[LS_idxs[u]].links = {
#      "": GRADESCOPE
# }

# LS_idxs[u] += 1


# date_ctx = MEETINGS[3]





# QZ[0].links = {
#     "What to Expect": "/resources/quiz-expectations.html",
#     "Practice Problems": "/resources/practice/sp24/qz00.html",
#     "Covered Topics: Syllabus + Units 0 and 1": "https://comp110-24s.github.io/resources/curriculum.html",
#     "Virtual Review Session (Passcode: 9&2MG!vk)": "https://unc.zoom.us/rec/share/Qn_n5MFAZUVtxLgqjSIv1eI_ueeCMNgLb0xMYf46gZ3aoQtod6mzxXnsrvzG0Ed4.pZNy8kPYnXDOzDav"
# }



# EX.append(schedule("Simple Battleship", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta + timedelta(days=5) ))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/simple-battleship.html"
# }
# EX_idxs[u] += 1

# VL.append(schedule("Virtual Lesson 00", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL00.html",
# }
# lessons = [["Conditionals", { "Video": "https://youtu.be/1OBa8SKWK40"}],
#            ["Boolean Operators", { "Video": "https://youtu.be/tmmSlIq9I_0", "Assignment" : GRADESCOPE}]]
# challenge_questions = []
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)

# date_ctx = MEETINGS[4]

# VL.append(schedule("Virtual Lesson 01", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL01.html",
# }
# lessons = [["Memory Diagrams", { "Video": "https://youtu.be/YKVT9mCtYPE", "Slides": "/static/slides/lessons/MemDiagrams.pdf"}],
#            ["Unicode, Emoji, Escape Sequences, and f-Strings", {"Reading": "/lessons/strings.html", "Assignment": GRADESCOPE}],
#            ["Relative Reassignment Operators", {"Video": "https://www.youtube.com/watch?v=GOs7pwPLB1k"}]]
# challenge_questions = [["Practice Memory Diagram",{"Assignment" : "https://www.gradescope.com/"}], ["Practice Memory Diagram with Conditionals",{"Assignment" : "https://www.gradescope.com/"}]]
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)

# CQ.append(schedule("Practice Memory Diagram", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": GRADESCOPE
# }
# CQ_idxs[u] += 1

# CQ.append(schedule("Practice Memory Diagram with Conditionals", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": GRADESCOPE
# }
# CQ_idxs[u] += 1


# date_ctx = MEETINGS[5]

# CL.append(schedule("Practice with Memory Diagrams and Conditionals ", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/LWPexWtwTkI?feature=share",
#     "Lecture Slides": "/static/slides/CL03-MDs.pdf"
# }
# CL_idxs[u] += 1

In [54]:
# date_ctx = MEETINGS[7]

# VL.append(schedule("Virtual Lesson", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL02.html",
# }
# lessons = [["while Loop Statements", { "Video": "https://youtu.be/iiksOU66xSw?si=qw0hZPiERbCJfkgt", "Slides": "/static/slides/while-loops.pdf", "Assignment": GRADESCOPE}], 
#            ["elif", {"Video": "https://www.youtube.com/watch?v=VIh0hc7ktE4", "Slides": "/static/slides/elif.pdf", "Assignment": GRADESCOPE}],
#            ["counters", {"Video": "https://youtu.be/XQPFd1Gae9M?si=erOJqqgKJhkUyntG"}]]
# make_VL_page(lessons,[], VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])

# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)



# date_ctx = MEETINGS[8]

# CL.append(schedule("Practice with while Loops", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/IzVtEvML7tY?feature=share",
#     "Lecture Slides": "/static/slides/CL04.pdf"
# }
# CL_idxs[u] += 1

# EX.append(schedule("One-shot Battleship", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/one-shot-battleship.html"
#     # "LS10 - Unicode, Emoji, Escape Sequences, and f-Strings": "https://22f.comp110.com/lessons/strings.html"
# }
# EX_idxs[u] += 1


# date_ctx = MEETINGS[9]

# CL.append(schedule("while Loops + Debugger", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/1V1LKJ0ETd0?feature=share"
# }
# CL_idxs[u] += 1

# CQ.append(schedule("while Loops + Debugger", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta + timedelta(days=4)))
# CQ[CQ_idxs[u]].links = {
#     "": GRADESCOPE
# }
# CQ_idxs[u] += 1


# RD.append(schedule("Reading: The Ethical Algorithm", unit[u], Kind.RD, RD_idxs[u], date_ctx, date_ctx + RD_due_delta))
# RD[RD_idxs[u]].links = {
#     "": GRADESCOPE
# }
# RD_idxs[u] += 1

# date_ctx = MEETINGS[10]

# VL.append(schedule("Virtual Lesson", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL03.html",
# }
# lessons = [["Introducing Functions", { "Video": "https://youtu.be/UAqFF9e6SLs", "Slides": "/static/slides/functions.pdf", "Assignment": GRADESCOPE}], 
#            ["Function Syntax", {"Video": "https://youtu.be/gOnn7OQhaj0", "Assignment": GRADESCOPE}],
#            ["Function Semantics", {"Video": "https://youtu.be/2RnI4bYMZww", "Assignment": GRADESCOPE}]]
# make_VL_page(lessons,[], VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])

# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)



# date_ctx = MEETINGS[11]

# CL.append(schedule("Practice With Functions", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/P7qTE-9_bGo?feature=share",
#     "Slides" : "/static/slides/CL06.pdf"
# }
# CL_idxs[u] += 1


# EX.append(schedule("Functional Battleship", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta))
# EX[EX_idxs[u]].links = {
#      "": "/exercises/battleship.html"
# }
# EX[EX_idxs[u]].add_extension(timedelta(days = 5))
# EX_idxs[u] += 1




# date_ctx = MEETINGS[12]

# #passcode = 
# #input = re.sub(r'\b'+elem+r'\b(?!@)', '#'+elem, input)

# QZ[1].links = {
#     "What to Expect": "/resources/quiz-expectations.html",
#     "Practice Problems": "/resources/practice/sp24/qz01.html",
#     "Covered Topics: Units 0-3B": "/resources/curriculum.html",
#     "Virtual Review Session (Passcode: a#6@NF+$)": "https://unc.zoom.us/rec/share/m1wp9lY_pIzm4Rp1RSVLmn3ewnebFuuEgonU8DP0Gb-z3eI4_WosMR-1QBAkLYX9.eD191e6op8lTWsFD"
# }










In [55]:
# date_ctx = MEETINGS[13]


# CL.append(schedule("Lists", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/hGcGYAkLa8A?feature=share",
#     "Slides" : "/static/slides/CL07.pdf"
# }
# CL_idxs[u] += 1

# LS_idxs[u] = len(LS_VIDEOS) 

# LS[LS_idxs[u]] =(schedule("Lists", unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + LS_due_delta + timedelta(days=1)))
# LS[LS_idxs[u]].links = {
#      "": GRADESCOPE
# }

# video = ["lists", { "Video": "https://youtu.be/U_GP3Jkz5lU", "sldies": "/static/slides/CL07.pdf"}]
# LS_VIDEOS.append(video)

# LS_idxs[u] += 1

# date_ctx = MEETINGS[14]

# CL.append(schedule("Lists Practice", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/1ZxYIt6vKlg?feature=share",

# }
# CL_idxs[u] += 1

# CQ.append(schedule("Lists + Functions", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/lists.html"
# }

# CQ[CQ_idxs[u]].add_extension(timedelta(days=3))
# CQ_idxs[u] += 1

# date_ctx = MEETINGS[15]

# VL.append(schedule("Virtual Lesson", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL04.html",
# }
# lessons = [["Lists in Memory", { "Video": "https://youtu.be/WCvrVTUZ-Ac", "Assignment": GRADESCOPE}], 
#            ["Importing Modules", {"Video": "https://youtu.be/-Q-9HmzEb38", "Assignment": GRADESCOPE}]]
# make_VL_page(lessons,[], VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])

# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)




# EX.append(schedule("List Utils", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta + timedelta(days=2)))
# EX[EX_idxs[u]].links = {
#      "": "/exercises/list-utils.html"
# }
# EX_idxs[u] += 1

# date_ctx = MEETINGS[16]

# CL.append(schedule("for loops", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/U6lfZU9q668?feature=share",
#     "Slides": "/static/slides/CL09.pdf"

# }
# CL_idxs[u] += 1

# date_ctx = MEETINGS[17]

# CL.append(schedule("for ... in range(...) loops", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/yM12Oxam8Og?feature=share",
#     "Slides": "/static/slides/CL10.pdf"
# }
# CL_idxs[u] += 1

# CQ.append(schedule("For Loops", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/for-loops.html"
# }
# CQ_idxs[u] += 1

# RD.append(schedule("Weapons of Math Destruction", unit[u], Kind.RD, RD_idxs[u], date_ctx, date_ctx + RD_due_delta))
# RD[RD_idxs[u]].links = {
#     "": GRADESCOPE
# }
# RD[RD_idxs[u]].add_extension(timedelta(days = 4))
# RD_idxs[u] += 1

# date_ctx = MEETINGS[18]

# VL.append(schedule("Virtual Lesson", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL05.html",
# }
# lessons = [["Dictionaries", { "Video": "https://www.youtube.com/watch?v=PHkFp_T--yk", "Slides": "/static/slides/LS19.pdf","Assignment": GRADESCOPE}], 
#            ["Dictionaries in Memory", {"Video": "https://youtu.be/tlfJfLBsuX0", "Assignment": GRADESCOPE}]]
# make_VL_page(lessons,[], VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])

# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)

# date_ctx = MEETINGS[19]

# CL.append(schedule("Dictionaries + for loops", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/VUo2ikqmVgI?feature=share",
#     "Slides": "/static/slides/CL11.pdf"
# }
# CL_idxs[u] += 1


# CQ.append(schedule("Dictionaries + for Loops", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta + timedelta(days=4)))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/dicts-for-loops.html"
# }
# CQ_idxs[u] += 1

# EX.append(schedule("Dictionary Utils", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/dict-utils.html"
# }
# EX_idxs[u] += 1

# # EX.append(schedule("Dictionary Unit Tests", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta))
# # EX[EX_idxs[u]].links = {
# #     "": "/exercises/dict_unit_tests.html"
# # }
# # EX_idxs[u] += 1




# # CL.append(schedule("Classes in Memory + Magic Methods", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# # CL[CL_idxs[u]].links = {
# #     "Lecture Video": "https://youtube.com/live/8F5zSJ-ctvU?feature=share",
# #     "Lecture Slides": "/static/slides/f23/CL10.pdf"
# # }
# # CL_idxs[u] += 1

# date_ctx = MEETINGS[20]

# QZ[2].links = {
#     "Practice Problems": "/resources/practice/sp24/qz02.html",
#     "Covered Topics: Units 0 - 4C": "https://comp110-24s.github.io/resources/curriculum.html",
#     "Virtual Review Session (Passcode: 3bQcXYz*)": "https://unc.zoom.us/rec/share/uJpFoC0a4ROk1eSU4baPxbtF2whm4iPVA_T7HPHPi5_1tQ2XaQc1hSfZS8z5DW6B.Lo_WpUq9SZfigTnY"
# }



In [56]:
# date_ctx = MEETINGS[21]

# VL.append(schedule("Virtual Lesson", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL06.html",
# }
# lessons = [["Unit Tests", { "Video": "https://youtu.be/d4RRQmNg5s4", "Assignment": GRADESCOPE}]]
# make_VL_page(lessons,[], VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u])

# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)


# date_ctx = MEETINGS[22]

# CL.append(schedule("Practice Writing Functions", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/PvigMl-M4i8?feature=share",
#     "Instructions": "/lessons/dict-practice.html"
# }
# CL_idxs[u] += 1

# EX.append(schedule("Dictionary Unit Tests", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta + timedelta(days = 9)))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/dict-utests.html"
# }

# EX_idxs[u] += 1


# date_ctx = MEETINGS[23]

# CQ.append(schedule("Dictionary Unit Tests Practice", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/dict_unit_tests.html"
# }
# CQ[CQ_idxs[u]].add_extension(timedelta(days = 11))
# CQ_idxs[u] += 1

# date_ctx = MEETINGS[24]

# VL.append(schedule("Virtual Lesson 07", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL07.html",
# }
# lessons = [["Mutable vs. Immutable Objects", { "Video": "https://youtu.be/izMVYxP7pPA", "Slides": "/static/slides/MutableObjects.pdf", "Assignment": GRADESCOPE}],
#            ["More on Unit Tests", {"Video": "https://youtu.be/_WNcQzdTvxM?si=SeJ--BxpOwjo7Qlh"}]]
# challenge_questions = [["Dictionary Unit Tests Practice",{"Assignment" : "/cqs/dict_unit_tests.html"}]]
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u]-1)
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)

# date_ctx = MEETINGS[25]

# CL.append(schedule("Big-O Notation", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/qpxq1ZL43EU?feature=share",
#     "Lecture Slides": "/static/slides/CL13.pdf"
# }
# CL_idxs[u] += 1

# date_ctx = MEETINGS[26]

# CL.append(schedule("More on Big-O Notation + Search Algorithms", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/sNBSqW2RWCE?feature=share",
#     "Lecture Slides": "/static/slides/CL14.pdf"
# }
# CL_idxs[u] += 1

# EX.append(schedule("Runtime Analysis", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta + timedelta(days = 5)))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/runtime.html"
# }
# EX_idxs[u] += 1

# RD.append(schedule("Reading: Man versus Machine or Man + Machine?", unit[u], Kind.RD, RD_idxs[u], date_ctx, date_ctx + RD_due_delta))
# RD[RD_idxs[u]].links = {
#     "": GRADESCOPE
# }
# RD_idxs[u] += 1







# date_ctx = MEETINGS[27]

# CL.append(schedule("VIRTUAL quiz review 2:30-4:25", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Recording (Passcode: n6fsTE=3)": "https://unc.zoom.us/rec/share/odtEUuVl-Oi5KJhBMBPTS7xvMqWd_iUkM1nBpDFa24w5lz9YhSm5wjkSLmbWjKh0.MNdE0B4PLlj6Pmkj"
# }
# CL_idxs[u] += 1


# QZ[3].links = {
#     "Practice Problems": "/resources/practice/sp24/qz03.html",
#     "Covered Topics: Units 0 - 5": "https://comp110-24s.github.io/resources/curriculum.html",
#     "Review Session Recording": "https://uncch.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=fd9a922d-7d6c-4839-a755-b13e0152dc4a"

#}




In [57]:
# date_ctx = MEETINGS[29]

# VL.append(schedule("Virtual Lesson 08", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL08.html",
# }
# lessons = [["Introduction to Recursion", { "Video Pt 1.": "https://youtu.be/5mVmXIrG5ec", "Video Pt. 2": "https://youtu.be/lWwCoD9EV9g","Slides": "/static/slides/Recursion.pdf", "Assignment" : GRADESCOPE}]]
# challenge_questions = []
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u]-1)
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)


# date_ctx = MEETINGS[30]

# CL.append(schedule("Reverse Engineering A Recursive Function", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/Aw3EPrnO6iQ?feature=share",
#     "Lecture Slides": "/static/slides/CL15.pdf"}
# CL_idxs[u] += 1

# EX.append(schedule("Recursive Turtle", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta + timedelta(days=2)))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/turtle.html"
# }
# EX_idxs[u] += 1
# date_ctx = MEETINGS[31]

# CL.append(schedule("Back to Defining A Function Recursively", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/lrmMM63lo9s"}
# CL_idxs[u] += 1

# CQ.append(schedule("Defining a Function Recursively", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/recursive-fns.html"
# }

# CQ_idxs[u] += 1


# date_ctx = MEETINGS[32]

# VL.append(schedule("Virtual Lesson 09", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL09.html",
# }
# lessons = [["Introduction to Object Oriented Programming", { "Video": "https://www.youtube.com/watch?v=DmYPEH8qkRI", "Slides": "/static/slides/OOP.pdf", "Assignment" : GRADESCOPE}]]
# challenge_questions = []
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u]-1)
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)

# date_ctx = MEETINGS[33]

# CL.append(schedule("Practice With OOP", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/bIuZTsTGp6k?feature=share",
#     "Slides": "/static/slides/OOP-Practice.pdf"}
# CL_idxs[u] += 1


# date_ctx = MEETINGS[34]

# CQ.append(schedule("Practice With OOP", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/oop-mutable.html"
# }

# CQ_idxs[u] += 1

# EX.append(schedule("River Simulation", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta + timedelta(days=5)))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/river-sim.html"
# }
# EX_idxs[u] += 1

# date_ctx = MEETINGS[35]

# VL.append(schedule("Virtual Lesson 10", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL10.html",
# }
# lessons = [["Magic Methods", { "Video": "https://www.youtube.com/watch?v=YvfnVMuwSEA", "Assignment" : GRADESCOPE}],
#            ["Operator Overload", { "Video": "https://www.youtube.com/watch?v=kMrRsmFzZBY", "Assignment" : GRADESCOPE}]]
# challenge_questions = []
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u]-1)
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)


# date_ctx = MEETINGS[36]

# CL.append(schedule("Practice With Magic Methods + Operator Overloads", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/QA3A0JxEhPM?feature=share",
#     "Slides": "/static/slides/MagicMethods.pdf"}
# CL_idxs[u] += 1


# date_ctx = MEETINGS[37]
# QZ[4].links = {
#     "Practice Problems": "/resources/practice/sp24/qz04.html",
#     "Covered Topics: Units 0 - 7": "https://comp110-24s.github.io/resources/curriculum.html",
#     "Review Session": "https://uncch.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=d3cd3d8f-698f-435f-bc16-b15700145bad"
# }


In [58]:
# date_ctx = MEETINGS[38]

# VL.append(schedule("Virtual Lesson 11", unit[u], Kind.VL, VL_idxs[u], date_ctx, date_ctx + VL_due_delta))
# VL[VL_idxs[u]].links = {
#     "": "/virtual-classes/VL11.html",
# }
# lessons = [["Recursive Structures", { "Video": "https://youtu.be/3WHUAPJmWKo", "slides": "/static/slides/RecursiveStructures.pdf","Assignment" : GRADESCOPE}]]
# challenge_questions = []
# make_VL_page(lessons,challenge_questions, VL_idxs[u],len(LS_VIDEOS),CQ_idxs[u]-1)
# VL_idxs[u] += 1
# LS_VIDEOS.extend(lessons)

# date_ctx = MEETINGS[39]

# CL.append(schedule("Recursive Structures + EX10 Prep", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://youtube.com/live/l6mpmswAKjk?feature=share"}
# CL_idxs[u] += 1

# CQ.append(schedule("Recursive Structures Node Class", unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + CQ_due_delta + timedelta(days=2)))
# CQ[CQ_idxs[u]].links = {
#     "": "/cqs/recursion.html"
# }

# CQ_idxs[u] += 1



# EX.append(schedule("Linked Lists", unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + EX_due_delta))
# EX[EX_idxs[u]].links = {
#     "": "/exercises/linked-list-utils.html"
# }
# EX_idxs[u] += 1

# date_ctx = MEETINGS[40]

# CL.append(schedule("Practice with OOP", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Practice Problem": "/resources/practice/sp24/fn-practice-prob.html", 
#     "Practice Diagram": "/resources/practice/memory-diagrams/oop-advanced/board-games.html"
#     }
# CL_idxs[u] += 1

# date_ctx = MEETINGS[41]

# CL.append(schedule("(Virtual) Final Exam Review", unit[u], Kind.CL, CL_idxs[u], date_ctx))
# CL[CL_idxs[u]].links = {
#     "Lecture Video": "https://uncch.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=b6408b4e-29a9-465c-8554-b16101555ef5"}
# CL_idxs[u] += 1

# FN[0].links = {
#     "Practice Problems": "/resources/practice/sp24/fn.html",
#     "Review Session Recording": "https://uncch.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=b6408b4e-29a9-465c-8554-b16101555ef5"
# }

# ## Other videos to add
# lessons = [["Dictionaries", { "Video": "https://youtu.be/PHkFp_T--yk"}],
#            ["Dictionaries in Memory", { "Video": "https://youtu.be/KK_gUoqducc"}],
#            ["Unit Tests", { "Video": "https://youtu.be/mxmAoCwVYLA"}],
#            ["Object Oriented Programming", { "Video": "https://youtu.be/DmYPEH8qkRI"}],
#             ["Classes in Memory", { "Video": "https://youtu.be/nJLXpxVafqg"}],
#             ["Magic Methods", { "Video": "https://youtu.be/YvfnVMuwSEA"}],
#             ["Operator Overloads", { "Video": "https://youtu.be/kMrRsmFzZBY"}],
#             ["Jupyter Notebooks", { "Video": "https://youtu.be/7-twq9rb4V8"}],
#             ["CSV Files", { "Video": "https://youtu.be/uWQpJ31C6u0"}],
#             ["More on Data", { "Video": "https://youtu.be/0eQC7Ivr-jQ"}]
            
            
# ]

# LS_VIDEOS.extend(lessons)

generate_VL_index_page(LS_VIDEOS)

In [59]:

syllabus = merge(syllabus, quizzes)
for u in unit:
    syllabus = merge(syllabus, u)
serializable_syllabus = serialize(syllabus)
show_calendar(syllabus)

Date,Weekday,Plan
Week 0,,
May-15,Wed,FDOC
,,Day 00
,,Welcome to COMP110 - An Intro to the Class
,,Out: Syllabus and Support
,,Out: Hello World
,,Due: Syllabus and Support
May-16,Thu,Drop Period Ends
,,Widthdraw Period Ends
,,Day 01


In [60]:
syllabus_copy = syllabus.copy()